In [1]:
using JuMP

In [19]:
# Define model
m = Model()

# Road/Snow removal machine combinations
S = ["1","2","3","4","5","6"]

# Non-negativity and binary constraint
@variable(m, x[S], Bin)

# Objective function: maximize
@objective(m, Max, 1000000x["1"] + 200000x["2"] + 300000x["3"] + 400000x["4"] + 450000x["5"] + 450000x["6"])

# Constraints
# Resource Constraints
# Cost
@constraint(m, 500000x["1"] + 150000x["2"] + 300000x["3"] + 250000x["4"] + 250000x["5"] + 100000x["6"] <= 1800000)
# Designers
@constraint(m, 700x["1"] + 250x["2"] + 200x["3"] + 200x["4"] + 300x["5"] + 400x["6"] <= 1500)
# Salesmen
@constraint(m, 200x["1"] + 100x["2"] + 100x["3"] + 100x["4"] + 100x["5"] + 1000x["6"] <= 1200) 
# Logical constraints
@constraint(m, x["6"] <= x["4"] + x["5"])
@constraint(m, x["2"] + x["5"] <= 1)

# Solve the optimazation problem
solve(m)

# Variable outcomes
println("Variable values: ", getvalue(x))

# Optimal objective value
println("Objective value: ", getobjectivevalue(m))

Variable values: x: 1 dimensions:
[1] = 1.0
[2] = 0.0
[3] = 1.0
[4] = 1.0
[5] = 1.0
[6] = 0.0

Objective value: 2.15e6


In [37]:
# Define model
light = Model()

# Intersections of corridors
C = ["1","2","3","4","5"]

# Create a binary variable for light fixtures and a continuous variable for light utility
@variable(light, y[C], Bin)
@variable(light, x[C] >= 0)
# Binary variable for light fixture at intersection 1
@variable(light, w, Bin)
# Binary variable for light fixtures less than 150 utility
@variable(light, z[C], Bin)

# Objective value to minimize cost
I = i=1:size(C)[1]
@objective(light, Min, 450*sum{y[C[i]], i=I}+2*sum{x[C[i]], i=I})

# Constraints
@constraint(light, x[C[1]] + x[C[2]] >= 250)
@constraint(light, x[C[1]] + x[C[3]] >= 300)
@constraint(light, x[C[2]] + x[C[3]] >= 150)
@constraint(light, x[C[2]] + x[C[4]] >= 200)
@constraint(light, x[C[3]] + x[C[5]] >= 350)
@constraint(light, x[C[4]] + x[C[5]] >= 180)
@constraint(light, iter[i=I], x[C[i]] <= 300*y[C[i]])
@constraint(light, x[C[1]] >= 300*w)
@constraint(light, x[C[4]] >=300*(1-w))
@constraint(light, iter[i=I], x[C[i]] <= 150 + 150*(1-z[C[i]]))
@constraint(light, sum{z[C[i]], i=I} >=3)
@constraint(light, 0.6*x[C[1]] - 0.4*x[C[3]] <= 0)


# Solve the optimization problem
solve(light)

# Variable outcomes
println("Variable values: ", getvalue(x))
println("Variable values: ", getvalue(y))

# Optimal objective value
println("Objective value: ", getobjectivevalue(light))

Variable values: x: 1 dimensions:
[1] = 150.00000000000003
[2] = 99.99999999999993
[3] = 300.0
[4] = 300.0
[5] = 50.00000000000001

Variable values: y: 1 dimensions:
[1] = 0.9999999999999999
[2] = 1.0
[3] = 1.0
[4] = 1.0
[5] = 1.0

Objective value: 4050.0


In [133]:
# Define model
factory = Model()

# Item day combinations
I = ["1","2","3"]
Item = i=1:size(I)[1]
# Number of days
T = ["0","1","2","3","4"]
Day = i=2:size(T)[1]

# Create a binary variable for light fixtures and a continuous variable for light utility
# Cost of inventory storage
@variable(factory, h[I])
# Time machine spends on item i on day t
@variable(factory, x[I,T] >= 0)
# Number of tiems produced on day t
@variable(factory, y[I,T] >= 0)
# Binary variable for whether the machine is set up for item i
@variable(factory, z[I,T], Bin)
# Amount of item i in inventory at the end of day t
@variable(factory, s[I,T] >= 0)
# Demand for item i on day t
d = [[0,400,600,200,800],[0,240,440,100,660],[0,80,120,40,100]]
# Production rate for item i
r = [5,4,2]
# Super-production days
@variable(factory, u[T], Bin)

# Objective function
a = [50,70,120]
b = [18,15,25]
@objective(factory, Max, sum{a[i]d[i][t] - b[i]s[I[i],T[t]], i=Item, t=Day} - 50*sum{u[T[t]], t=Day})

# Constraints
# Machine usage time
@constraint(factory, days[t=Day], 45*z[I[1],T[t]] + 60*z[I[2],T[t]] + 100*z[I[3],T[t]] + sum{x[I[i],T[t]], i=Item} <= 480 + 120*u[T[t]])
# Inventory at end of day constraints
@constraint(factory, dayitem[t=Day,i=Item], s[I[i],T[t-1]] + y[I[i],T[t]] - d[i][t] == s[I[i],T[t]])
@constraint(factory, item[i=Item], s[I[i],T[1]] == 0)
@constraint(factory, item[i=Item], s[I[i],T[5]] == 0)
# Additional constraints
M = 600
@constraint(factory, dayitem[t=Day,i=Item], x[I[i],T[t]] <= M*z[I[i],T[t]])
@constraint(factory, dayitem[t=Day,i=Item], y[I[i],T[t]] == r[i]*x[I[i],T[t]])
@constraint(factory, sum{u[T[t]], t=Day} <= 2)
@constraint(factory, day[t=2:5], u[T[t-1]] + u[T[t]] <= 1)


# Solve the optimization problem
solve(factory)

# Variable outcomes
println("Variable values x: ", getvalue(x))
println("Variable values y: ", getvalue(y))
println("Variable values z: ", getvalue(z))
println("Variable values s: ", getvalue(s))
println("Variable values u: ", getvalue(u))

# Optimal objective value
println("Objective value: ", getobjectivevalue(factory))

Variable values x: x: 2 dimensions:
[1,:]
  [1,0] = 0.0
  [1,1] = 79.99999999999999
  [1,2] = 120.0
  [1,3] = 40.0
  [1,4] = 159.99999999999997
[2,:]
  [2,0] = 0.0
  [2,1] = 60.0
  [2,2] = 110.00000000000001
  [2,3] = 25.0
  [2,4] = 165.00000000000003
[3,:]
  [3,0] = 0.0
  [3,1] = 40.0
  [3,2] = 60.0
  [3,3] = 20.0
  [3,4] = 50.00000000000001

Variable values y: y: 2 dimensions:
[1,:]
  [1,0] = 0.0
  [1,1] = 400.00000000000006
  [1,2] = 600.0
  [1,3] = 200.00000000000003
  [1,4] = 800.0000000000001
[2,:]
  [2,0] = 0.0
  [2,1] = 240.0
  [2,2] = 440.0
  [2,3] = 99.99999999999999
  [2,4] = 660.0
[3,:]
  [3,0] = 0.0
  [3,1] = 80.0
  [3,2] = 119.99999999999999
  [3,3] = 39.99999999999999
  [3,4] = 99.99999999999999

Variable values z: z: 2 dimensions:
[1,:]
  [1,0] = 0.0
  [1,1] = 1.0
  [1,2] = 1.0
  [1,3] = 1.0
  [1,4] = 1.0
[2,:]
  [2,0] = 0.0
  [2,1] = 1.0
  [2,2] = 1.0
  [2,3] = 1.0
  [2,4] = 1.0
[3,:]
  [3,0] = 0.0
  [3,1] = 1.0
  [3,2] = 1.0
  [3,3] = 1.0
  [3,4] = 1.0

Variable value

In [160]:
# Define model: solving a sudoku puzzle
sudoku = Model()

# Number of rows
I = ["1","2","3","4","5","6","7","8","9"] 
Row = i=1:size(I)[1]
# Number of columns
J = ["1","2","3","4","5","6","7","8","9"]
Column = j=1:size(J)[1]
# Numbers
K = ["1","2","3","4","5","6","7","8","9"]
Cell = k=1:size(K)[1]

# Create a binary variable which equals 1 if we put number k 
# in the ith row and jth column and 0 otherwise
@variable(sudoku, x[I,J,K], Bin)
# set up the initial puzzle
a = zeros(Int64,9,9,9)
a[1,1,5] = 1
a[1,2,3] = 1
a[1,5,7] = 1
a[2,1,6] = 1
a[2,4,1] = 1
a[2,5,9] = 1
a[2,6,5] = 1
a[3,2,9] = 1
a[3,3,8] = 1
a[3,8,6] = 1
a[4,1,8] = 1
a[4,5,6] = 1
a[4,9,3] = 1
a[5,1,4] = 1
a[5,4,8] = 1
a[5,6,3] = 1
a[5,9,1] = 1
a[6,1,7] = 1
a[6,5,2] = 1
a[6,9,6] = 1
a[7,2,6] = 1
a[7,7,2] = 1
a[7,8,8] = 1
a[8,4,4] = 1
a[8,5,1] = 1
a[8,6,9] = 1
a[8,9,5] = 1
a[9,5,8] = 1
a[9,8,7] = 1
a[9,9,9] = 1


# Sudoku is a feasibility problem, so the objective is always 0.
@objective(sudoku, Max, 0)

# Constraints
@constraint(sudoku, ijk[i=Row,j=Column,k=Cell], x[I[i],J[j],K[k]] >= a[i,j,k])
# Each row must contain each of the integers in [1,9]
@constraint(sudoku, jk[j=Column,k=Cell], sum{x[I[i],J[j],K[k]],i=Row} == 1)
# Each column must contain each of the integers in [1,9]
@constraint(sudoku, ik[i=Row,k=Cell], sum{x[I[i],J[j],K[k]],j=Column} == 1)
# Each cell contains an integer in [1,9]
@constraint(sudoku, ij[i=Row,j=Column], sum{x[I[i],J[j],K[k]],k=Cell} == 1)
# Each of the nine 3x3 squares with bold outlines must contain each of the integer in [1,9]
@constraint(sudoku, ijk[i=0:2,j=0:2,k=Cell], sum{x[I[3*i+b],J[3*j+c],K[k]],b=1:3,c=1:3} == 1)

# Solve the optimization problem
solve(sudoku)

# Variable outcomes
println("Variable values x: ", getvalue(x))

# Optimal objective value
println("Objective value: ", getobjectivevalue(sudoku))


Variable values x: x: 3 dimensions:
[1,:,:]
  [1,1,:]
    [1,1,1] = 0.0
    [1,1,2] = 0.0
    [1,1,3] = 0.0
    [1,1,4] = 0.0
    [1,1,5] = 1.0
    [1,1,6] = 0.0
    [1,1,7] = 0.0
    [1,1,8] = 0.0
    [1,1,9] = 0.0
  [1,2,:]
    [1,2,1] = 0.0
    [1,2,2] = 0.0
    [1,2,3] = 1.0
    [1,2,4] = 0.0
    [1,2,5] = 0.0
    [1,2,6] = 0.0
    [1,2,7] = 0.0
    [1,2,8] = 0.0
    [1,2,9] = 0.0
  [1,3,:]
    [1,3,1] = 0.0
    [1,3,2] = 0.0
    [1,3,3] = 0.0
    [1,3,4] = 1.0
    [1,3,5] = 0.0
    [1,3,6] = 0.0
    [1,3,7] = 0.0
    [1,3,8] = 0.0
    [1,3,9] = 0.0
  [1,4,:]
    [1,4,1] = 0.0
    [1,4,2] = 0.0
    [1,4,3] = 0.0
    [1,4,4] = 0.0
    [1,4,5] = 0.0
    [1,4,6] = 1.0
    [1,4,7] = 0.0
    [1,4,8] = 0.0
    [1,4,9] = 0.0
  [1,5,:]
    [1,5,1] = 0.0
    [1,5,2] = 0.0
    [1,5,3] = 0.0
    [1,5,4] = 0.0
    [1,5,5] = 0.0
    [1,5,6] = 0.0
    [1,5,7] = 1.0
    [1,5,8] = 0.0
    [1,5,9] = 0.0
  [1,6,:]
    [1,6,1] = 0.0
    [1,6,2] = 0.0
    [1,6,3] = 0.0
    [1,6,4] = 0.0
    [1,6,5] = 